In [1]:
import numpy as np
import pandas as pd
import datetime

In [2]:
link_df = pd.read_csv('../../data/original/links (table 3).csv', index_col=0)

In [3]:
# validation_path = '../../data/offline/validation_9_6/'
# window_date = pd.to_datetime('2016-09-06')
# prediction_file_name = 'prediction_9_6'

# validation_path = '../../data/offline/validation_9_20/'
# window_date = pd.to_datetime('2016-09-20')
# prediction_file_name = 'prediction_9_20'

validation_path = '../../data/offline/validation_10_11/'
window_date = pd.to_datetime('2016-10-11')
prediction_file_name = 'prediction_10_11'

In [4]:
def ArrayMape(ground_truth, predictions):
    diff = (ground_truth - predictions) / ground_truth
    diff = np.abs(diff)
    mape = diff.sum() / ground_truth.size
    
    return mape

In [5]:
def ScalarMape(ground_truth, predictions):
    diff = (ground_truth - predictions) / ground_truth
    diff = np.abs(diff)
    mape = diff
    
    return mape

In [32]:
for l in range(100, 124):
    print(l)
    print(afternoon_predict_dict[l])

100
          0
0  7.636523
1  7.646486
2  7.523470
3  7.417486
4  7.962809
5  7.331027
6  7.374264
101
           0
0   9.432891
1   9.446046
2   9.432891
3  11.133607
4  10.636389
5   9.950778
6  10.809649
102
           0
0  10.476280
1  10.660764
2  10.845312
3  10.492794
4  10.406298
5  10.424958
6  10.604580
103
          0
0  4.416196
1  4.487507
2  3.705297
3  3.639282
4  3.770513
5  3.768450
6  4.453500
104
           0
0  28.234318
1  32.838304
2  28.445234
3  35.667132
4  36.588199
5  29.307108
6  31.429779
105
           0
0  12.500920
1  11.690594
2  11.756579
3  11.701923
4  12.108477
5  12.365650
6  11.067562
106
          0
0  1.753932
1  2.004971
2  1.783981
3  1.689756
4  1.665685
5  1.914669
6  1.753233
107
          0
0  4.889992
1  4.911724
2  4.870199
3  4.846663
4  4.350770
5  4.150934
6  4.859462
108
          0
0  5.533323
1  5.592278
2  5.523159
3  5.776196
4  5.774810
5  5.004495
6  5.776196
109
           0
0  10.769256
1  11.063654
2  10.680266
3  12.370623

In [33]:
afternoon_predict_dict = {}
morning_predict_dict = {}

afternoon_real_dict = {}
morning_real_dict = {}

for link_id in link_df.index.values:
    predict_afternoon_df = pd.read_csv(validation_path + 'prediction/feature7/afternoon/' + str(link_id) + '.csv', index_col=0)
    predict_morning_df = pd.read_csv(validation_path + 'prediction/feature7/morning/' + str(link_id) + '.csv', index_col=0)
    
    real_afternoon_df = pd.read_csv(validation_path + 'feature/feature7/afternoon/y_test_link' + str(link_id) + '.csv', index_col = 0)
    real_morning_df = pd.read_csv(validation_path + 'feature/feature7/morning/y_test_link' + str(link_id) + '.csv', index_col = 0)

    
    afternoon_predict_dict[link_id] = predict_afternoon_df;
    morning_predict_dict[link_id] = predict_morning_df
    
    afternoon_real_dict[link_id] = real_afternoon_df
    morning_real_dict[link_id] = real_morning_df

In [34]:
routes = pd.read_csv('../../data/original/routes (table 4).csv')
submissions = []
for i in routes.index:
    link_seq = routes.loc[i]['link_seq']
    links = link_seq.split(',')
    
    #morning
    start_time = window_date + datetime.timedelta(hours=8)
    for d in range(7):
        current_time = start_time
        for t in range(6):
            predict_time = 0
            real_time = 0
            record = []
            record.append(routes.loc[i]['intersection_id'])
            record.append(routes.loc[i]['tollgate_id'])
            record.append('[' + str(current_time) + ',' + str(current_time + datetime.timedelta(minutes=20)) + ')')
            for link_id in links:
                predict_time = predict_time + morning_predict_dict[int(link_id)].loc[d][0]
                real_time = real_time + morning_real_dict[int(link_id)].loc[d][0]
                
            record.append(predict_time)
            record.append(real_time)

            submissions.append(record)
            current_time = current_time + datetime.timedelta(minutes=20)
            
        start_time = start_time + datetime.timedelta(days=1)
        
    #afternoon
    start_time = window_date + datetime.timedelta(hours=17)
#     start_time = pd.to_datetime('2016-09-20 17:00:00')
#     start_time = pd.to_datetime('2016-10-11 17:00:00')
    for d in range(7):
        current_time = start_time
        for t in range(6):
            predict_time = 0
            real_time = 0
            record = []
            record.append(routes.loc[i]['intersection_id'])
            record.append(routes.loc[i]['tollgate_id'])
            record.append('[' + str(current_time) + ',' + str(current_time + datetime.timedelta(minutes=20)) + ')')
            for link_id in links:
                predict_time = predict_time + afternoon_predict_dict[int(link_id)].loc[d][0]
                real_time = real_time + afternoon_real_dict[int(link_id)].loc[d][0]
                
            record.append(predict_time)
            record.append(real_time)

            submissions.append(record)
            current_time = current_time + datetime.timedelta(minutes=20)
            
        start_time = start_time + datetime.timedelta(days=1)

In [35]:
submission_df = pd.DataFrame(submissions).rename(columns={0:'intersection_id', 1:'tollgate_id', 2:'time_window', 3:'predict_time', 4:'real_time'})

In [36]:
mape_sum = 0
mapes = []
for i in range(len(submission_df)):
    #mape = ScalarMape(submission_df.loc[i]['real_time'], submission_df.loc[i]['predict_time'])
    mape = np.abs(submission_df.loc[i]['real_time'] - submission_df.loc[i]['predict_time'])
    
    #print('%a-%d : %f'%(m[0], m[1], mape))
    mapes.append(mape)
    mape_sum = mape_sum + mape
    
submission_df['mape'] = mapes
print("mape=%d"%(mape_sum))

mape=3693


In [20]:
mape=3537  #15
mape=3919  #25
mape=4109  #35
mape=4018  #40
mape=3915  #50
mape=4030  #80
mape=3869  #100


#9-6 mape=0.170663
#9-20 mape=0.115456
#mape=0.114156

In [76]:
submission_df['mape'].mean()

0.11415555656052243

In [78]:
submission_df[submission_df['mape'] <+ 0.17]['mape'].count()

399

In [26]:
submission_df.to_csv('../../data/offline/prediction/m1_m2/' + prediction_file_name + '.csv')

In [12]:
submission_df['mape'].describe()

count    504.000000
mean       0.179984
std        0.163925
min        0.000069
25%        0.064079
50%        0.140365
75%        0.252415
max        1.576950
Name: mape, dtype: float64

In [93]:
submission_df[submission_df['mape'] > 0.17]

,intersection_id,tollgate_id,time_window,predict_time,real_time,mape
0,A,2,"[2016-09-06 08:00:00,2016-09-06 08:20:00)",67.094925,47.851000,0.402163
1,A,2,"[2016-09-06 08:20:00,2016-09-06 08:40:00)",67.094925,54.396000,0.233453
2,A,2,"[2016-09-06 08:40:00,2016-09-06 09:00:00)",67.094925,51.218333,0.309979
3,A,2,"[2016-09-06 09:00:00,2016-09-06 09:20:00)",67.094925,53.237341,0.260298
4,A,2,"[2016-09-06 09:20:00,2016-09-06 09:40:00)",67.094925,49.496000,0.355563
8,A,2,"[2016-09-07 08:40:00,2016-09-07 09:00:00)",66.318603,47.293568,0.402275
11,A,2,"[2016-09-07 09:40:00,2016-09-07 10:00:00)",66.318603,55.040635,0.204903
12,A,2,"[2016-09-08 08:00:00,2016-09-08 08:20:00)",66.714030,53.747788,0.241242
16,A,2,"[2016-09-08 09:20:00,2016-09-08 09:40:00)",66.714030,55.655827,0.198689
20,A,2,"[2016-09-09 08:40:00,2016-09-09 09:00:00)",66.326138,80.249384,0.173500


In [17]:
anomaly_df = submission_df[submission_df['mape'] > 0.31]

In [18]:
anomaly_df.groupby(['intersection_id', 'tollgate_id']).count()

time_window  predict_time  real_time  mape
intersection_id tollgate_id                                            
A               2                     10            10         10    10
                3                     19            19         19    19
B               1                     13            13         13    13
                3                     22            22         22    22
C               1                      9             9          9     9
                3                     16            16         16    16

In [59]:
anomaly_df.groupby(['intersection_id', 'tollgate_id']).count()

time_window  predict_time  real_time  mape
intersection_id tollgate_id                                            
A               2                     17            17         17    17
                3                     17            17         17    17
B               1                     10            10         10    10
                3                     17            17         17    17
C               1                     12            12         12    12
                3                     14            14         14    14

In [41]:
anomaly_df.groupby(['intersection_id', 'tollgate_id']).count()

time_window  predict_time  real_time  mape
intersection_id tollgate_id                                            
A               2                     17            17         17    17
                3                     28            28         28    28
B               1                     22            22         22    22
                3                     25            25         25    25
C               1                     21            21         21    21
                3                     20            20         20    20

In [31]:
submission_df.sort_values('mape')

,intersection_id,tollgate_id,time_window,predict_time,real_time,mape
446,C,3,"[2016-09-10 08:40:00,2016-09-10 09:00:00)",166.776793,166.801628,0.000149
486,C,3,"[2016-09-10 17:00:00,2016-09-10 17:20:00)",181.657452,181.710298,0.000291
269,B,3,"[2016-09-08 09:40:00,2016-09-08 10:00:00)",100.343104,100.404368,0.000610
331,B,3,"[2016-09-12 17:20:00,2016-09-12 17:40:00)",101.994318,102.058818,0.000632
18,A,2,"[2016-09-09 08:00:00,2016-09-09 08:20:00)",78.898403,78.733785,0.002091
61,A,2,"[2016-09-09 17:20:00,2016-09-09 17:40:00)",71.478236,71.240714,0.003334
68,A,2,"[2016-09-10 17:40:00,2016-09-10 18:00:00)",71.014296,70.759273,0.003604
273,B,3,"[2016-09-09 09:00:00,2016-09-09 09:20:00)",118.138210,117.626503,0.004350
159,A,3,"[2016-09-11 18:00:00,2016-09-11 18:20:00)",116.436087,115.920543,0.004447
396,C,1,"[2016-09-09 17:00:00,2016-09-09 17:20:00)",206.390607,205.370278,0.004968


In [10]:
mape_dict = {}
mape_sum = 0
for link_id in link_df.index.values:
    morning_mape = ArrayMape(morning_real_dict[link_id][2].values, morning_predict_dict[link_id][0].values)
    afternoon_mape = ArrayMape(afternoon_real_dict[link_id][2].values, afternoon_predict_dict[link_id][0].values)
    print('morning:%d mape=%f'%(link_id, morning_mape))
    print('afternoon:%d mape=%f'%(link_id, afternoon_mape))
    mape_sum = mape_sum + morning_mape + afternoon_mape
    mape_dict[link_id] = [morning_mape, afternoon_mape]

print('mape=%f'% (mape_sum / link_df['length'].count() / 2))

morning:100 mape=0.182095
afternoon:100 mape=0.224775
morning:101 mape=0.185093
afternoon:101 mape=0.371319
morning:102 mape=0.151957
afternoon:102 mape=0.179695
morning:103 mape=0.468896
afternoon:103 mape=0.373826
morning:104 mape=0.173277
afternoon:104 mape=0.235542
morning:105 mape=0.275713
afternoon:105 mape=0.274362
morning:106 mape=0.326857
afternoon:106 mape=0.361005
morning:107 mape=0.203918
afternoon:107 mape=0.253931
morning:108 mape=0.196537
afternoon:108 mape=0.197880
morning:109 mape=0.137740
afternoon:109 mape=0.159067
morning:110 mape=0.204659
afternoon:110 mape=0.190731
morning:111 mape=0.471663
afternoon:111 mape=0.356867
morning:112 mape=0.359530
afternoon:112 mape=0.436725
morning:113 mape=0.307870
afternoon:113 mape=0.388915
morning:114 mape=0.302873
afternoon:114 mape=0.225402
morning:115 mape=0.204463
afternoon:115 mape=0.182797
morning:116 mape=0.128791
afternoon:116 mape=0.292142
morning:117 mape=0.210714
afternoon:117 mape=0.314066
morning:118 mape=0.451908
af

In [17]:
pd.DataFrame(mape_dict).T

,0,1
100,0.182095,0.224775
101,0.185093,0.371319
102,0.151957,0.179695
103,0.468896,0.373826
104,0.173277,0.235542
105,0.275713,0.274362
106,0.326857,0.361005
107,0.203918,0.253931
108,0.196537,0.197880
109,0.137740,0.159067
